### IMPORTS

In [1]:
from Inference import BBVI 

In [2]:
import torch
import matplotlib.pyplot as plt
import _pickle as pickle

### EXAMPLE

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

In [4]:
# load data
data = torch.load('C:\\Users\\cortax\\Documents\\BayesianNN\\data\\foong_data.pt')
x_data = data[0].to(device)
y_data = data[1].to(device)
y_data = y_data.unsqueeze(-1)

In [5]:
filename = 'C:\\Users\\cortax\\Documents\\BayesianNN\\Experiments\\foong\\tanh\\BBVI\\models\\' + '3Layers_50Neurons_(1)'
filehandler = open(filename, 'rb')
netparam = pickle.load(filehandler)

In [6]:
Net = BBVI.VariationalNetwork(input_size=netparam['input_size'],
                              output_size=netparam['output_size'],
                              layer_width=netparam['layer_width'],
                              nb_layers=netparam['nb_layers'])
Net.set_network(netparam)
Net.set_device(device)
Net.requires_grad_mus(False)

In [7]:
Net.compute_elbo(x_data, y_data, n_samples_ELBO=1000, sigma_noise=0.1, device=device).detach().cpu().numpy()

array(448.26526, dtype=float32)

In [ ]:
self = Net

In [ ]:
(layered_w_samples, layered_bias_samples) = self.sample_parameters(1000)

In [ ]:
mu = torch.cat([torch.cat((torch.flatten(self.registered_layers[k].prior_weight_mu), self.registered_layers[k].prior_bias_mu)) for k in range(len(self.registered_layers))])
mu.shape

In [ ]:
rho = torch.cat([torch.cat((torch.flatten(self.registered_layers[k].prior_weight_rho), self.registered_layers[k].prior_bias_rho)) for k in range(len(self.registered_layers))])
rho.shape

In [ ]:
sigma = self._rho_to_sigma(rho)

In [ ]:
nw = torch.distributions.MultivariateNormal(mu, scale_tril = torch.diag(sigma))

In [ ]:
w = torch.cat([torch.cat((torch.flatten(layered_w_samples[k], start_dim=1), layered_bias_samples[k]),dim=1) for k in range(len(self.registered_layers))], dim=1)

In [ ]:
w.shape

In [ ]:
LQ = self.q_log_pdf(layered_w_samples, layered_bias_samples)
LP = self.prior_log_pdf(layered_w_samples, layered_bias_samples)

In [ ]:
LQ.shape

In [ ]:
y_pred = self.forward(x_data)

In [ ]:
y_data.shape

In [ ]:
LL = self._log_norm(y_pred, y_data, torch.tensor(0.1).to(device))
LL.sum()

In [ ]:
.sum()